<a href="https://colab.research.google.com/github/isegura/OCW-UC3M-NLPDeep-2023/blob/main/tema5_5_tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png" width=50%/>

<h1><font color='#12007a'>Procesamiento de Lenguaje Natural con Aprendizaje Profundo</font></h1>
<p>Autora: Isabel Segura Bedmar</p>

<img align='right' src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>   

# 5.5. Tokenización en transformers

En este ejercicio, vamos a practicar con un tokenizador de un modelo tranformer.
En particular, vamos a trabajar con el tokenizador del modelo multilingüe BERT en su versión uncased (https://huggingface.co/bert-base-multilingual-uncased). Este modelo fue pre-entrenado con textos de wikipedia escritos en más de 102 idiomas, y por tanto, nos nos permitirá procesar textos en español. La versión 'uncased' significa que el modelo no distingue entre mayúsculas y minúsculas (recomendable para tareas de clasificación de textos).



El proceso de tokenización en un modelo transformer consiste en los siguientes pasos:

- Divide texto en tokens (palabras o partes de palabras, y signos de puntuación).
- Representa cada token con un número (índice del token en el vocabulario* del transformer).
- Cada token es transformado a un vector (tensor) de números reales.

* Vocabulario (vocab): conjunto de tokens de la colección de textos que fue utilizada para pre-entrenar el transformer.

<br>

Si vas usar un determinado modelo pre-entrenado (por ejemplo, BERT, GPT o XLNet), siempre deberás utlizar el tokenizador asociado a ese modelo:
- utilizar las mismas reglas para dividir el texto.
- usar los mismos índices que en el vocabulario del modelo.





## Instalación librería transformers

El primer paso será instalar la librería transformers de Hugging Face que nos permitirá cargar el modelo tranformer y su tokenizador.
La  instalaremos en modo silencioso (-q) para que no muestra las dependencias durante la instalación.


In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.6 MB/s eta 0:00:00


## Cargar tokenizador

Para cargar el tokenizador, usaremos la clase **AutoTokenizer**, a la que simplemente tendremos que indicarle en su método **from_pretrained** el tokenizador del modelo tranformer que queremos utilizar:


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-uncased')

Vamos a utilizar el tokenizador para procesar una oración en español:

In [ ]:
encoded = tokenizer("Hola, esta es una sencilla oración para ver la salida del tokenizador de BERT!")
encoded

{'input_ids': [101, 69887, 10112, 117, 10602, 10153, 10155, 12244, 51120, 14116, 10985, 10239, 15405, 10106, 40151, 10134, 16925, 13649, 20465, 10131, 10102, 40668, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Podemos ver que el tokenizador devuelve un diccionario con tres campos:

- **input_ids** que como valor asociado tiene una lista de números enteros. En concreto, son los índices de cada token en nuestra oración.
- **token_type_ids**, su valor asociado es una lista de 0's y 1's. 0 indica que el token pertenece a una primera oración, y 1 indica que pertenece a la segunda oración. En nuestro caso, como únicamente se ha procesado una oración, la lista únicamente contiene 0's. Veremos un ejemplo con dos oraciones en las siguientes celdas.

- **attention_mask**, su valor asociado también sería una lista de 0's y 1's. 0 representa a un token de padding, y 1 un token real de la oración. En nuestro ejemplo, como no hemos aplicado ningún padding, todos los elementos de la lista son 1.




In [ ]:
print("input_ids:", encoded.input_ids)
print("token_type_ids:", encoded.token_type_ids)
print("attention_mask:", encoded.attention_mask)

input_ids: [101, 69887, 10112, 117, 10602, 10153, 10155, 12244, 51120, 14116, 10985, 10239, 15405, 10106, 40151, 10134, 16925, 13649, 20465, 10131, 10102, 40668, 106, 102]
token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]



En las siguientes celdas, volveremos a estudiar con más detalles estas dos listas **token_type_ids** y **attention_mask**. Por el momento, nos centraremos únicamente en **input_ids**.

## input_ids

Vamos a procesar una oración aún más sencilla, formada por una única palabra: "Ven".

In [ ]:
encoded = tokenizer("Ven")
print("input_ids:", encoded.input_ids)
print("token_type_ids:", encoded.token_type_ids)
print("attention_mask:", encoded.attention_mask)

input_ids: [101, 22712, 102]
token_type_ids: [0, 0, 0]
attention_mask: [1, 1, 1]


Aunque únicamente contiene una palabra, el tokenizador devuelve una lista de tres ids: $[101, 22712, 102]$

Podemos decodificarla usando el método **convert_ids_to_tokens**, que transforma de enteros a palabras:

In [ ]:
tokenizer.convert_ids_to_tokens(encoded.input_ids)

['[CLS]', 'ven', '[SEP]']

Así podemos ver fácilmente:
- 101 es el id de CLS.
- 102 es el id de SEP.
- 22712 es el id de ven.



Por tanto, el tokenizador ha añadido automáticamente dos tokens especiales a la oración:
- [𝐶𝐿𝑆] indica el comienzo de la primera oración, y
- [𝑆𝐸𝑃] indica el final de cada oración.

Fijate que hizo lo mismo también en la primera oración.

## token_type_ids
Centremosno ahora en el campo **token_type_ids**. Al procesar la oración 'Ven' (o también la primera oración), este campo únicamente contiene una lista de 0's.



In [ ]:
print("token_type_ids:", encoded.token_type_ids)


token_type_ids: [0, 0, 0]



BERT fue entrenado con dos estrategias **Masked language modeling (MLM)** (el modelo tiene que aprende a predecir ciertos tokens que han sido ocultados a partir de los tokens de su contexto) y **Next Sentence Prediction** (NSP) (el modelo aprende a predecir si dos oraciones aparecen consecutivas en los textos de entrenamiento o no.

BERT va a utilizar el campo **token_type_ids** para la tarea de NSP. En los ejemplos anteriores, BERT únicamente estaba procesando una oración de entrada, y por esos todos sus tokens eran anotados como 0 en esta capa token_type_ids.
Veamos qué ocurre si la entrada es un par de oraciones:


In [ ]:
s1 = 'Hoy es miércoles.'
s2 = 'Mi día favorito es el viernes.'

encoded = tokenizer(s1,s2)

print("input_ids:", encoded.input_ids)
print("token_type_ids:", encoded.token_type_ids)
print("attention_mask:", encoded.attention_mask)

input_ids: [101, 21150, 10153, 95096, 66849, 11300, 119, 102, 11793, 10604, 19266, 12875, 10153, 10117, 94953, 119, 102]
token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


En este caso, sí podemos ver que la lista asociada con **token_type_ids*** es una lista de 0's y 1's, donde 0 indica que es un token de la primera oración, y 1 indica que es un token de la segunda oración.


La salida de los tokenizadores no es igual para todos los modelos.

Por ejemplo, el tokenizador asociado al modelo **gpt2-medium**, cuando procesa una oración, devueve una lista de input_ids y también attention_mask, pero no devuelve token_type_ids.

¿Por qué esta diferencia?. El modelo GPT2, a diferencia de BERT, no fue entrenado con la tarea NSP, y por tanto, no necesita distinguir si un token es de una oración o de la siguiente.

Vamos a comparar como los dos tokenizadores, de BERT y GPT2, procesan una misma oración:

In [ ]:
tokenizer_gpt2 = AutoTokenizer.from_pretrained('gpt2-medium')
encoded_gpt2 = tokenizer_gpt2("Hello, I'm a single sentence!")
print(encoded_gpt2)

{'input_ids': [15496, 11, 314, 1101, 257, 2060, 6827, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
encoded_bert = tokenizer("Hello, I'm a single sentence!")
print(encoded_bert)

{'input_ids': [101, 29155, 117, 151, 112, 155, 143, 11304, 45261, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Vemos que la codificación por BERT sí tiene el campo **token_type_ids**, mientras que la tokenización proporcionada por el tokenizador de gpt2, no contiene dicho campo.


Otra diferencia es que este tokenizador no añade tampoco los tokens especiales [𝐶𝐿𝑆] y [SEP], que si eran añadidos por el tokenizador de BERT.

In [ ]:
tokenizer_gpt2.convert_ids_to_tokens(encoded_gpt2.input_ids)

['Hello', ',', 'ĠI', "'m", 'Ġa', 'Ġsingle', 'Ġsentence', '!']

In [ ]:
tokenizer.convert_ids_to_tokens(encoded_bert.input_ids)

['[CLS]', 'hello', ',', 'i', "'", 'm', 'a', 'single', 'sentence', '!', '[SEP]']

En el próximo ejercicio, aprenderemos a tokenizar una colección de textos y revisaremos los proceso de padding y truncation, y como son aplicados en la tokenización de los transformers.



## Wordpiece tokenization

Cuando un transformer es pre-entrenado, una de las tareas que se realiza es construir un vocabulario formado por todas las palabras distintas que ocurren la colección de textos utilizada para entrenar el modelo.

En ese vocabulario cada token (palabra o signo de puntuación) está asociada con un índice que permite representar de forma única a cada token.

Posteriormente, el transformer puede ser ajustado (fine-tuned) con un dataset específico para una tarea concreta de PLN (por ejemplo, la clasificación de textos). Los textos de dicho dataset serán tokenizados utilizando el tokenizador que fue pre-entrenado junto con el transformer. Pero ¿qué ocurre si en dichos textos ocurren palabras que no están en el vocabulario del tokenizador?

El tokenizador de BERT, y otros modelos transformers, incluyen también en su vocabulario los morfemas y subpalabras más frecuentes. Entonces, las palabras que no están en el vocabulario pueden ser divididas en sus morfemas, y representadas con sus correspondientes indentificadores en el vocabulario aprendido durante el pre-entrenamiento de BERT.   


Si estás interesado en el método utilizado para entrenar este tokenizador, conocido como **wordpiece**, consulta encontrar más información en este enlace https://huggingface.co/course/chapter6/6?fw=pt.

En este ejercicio, vamos a ver simplemente como el tokenizador de BERT tokeniza algunas oraciones donde ocurren palabras que no están presentes en el vocabulario del modelo.

Vamos a cargar ahora el modelo BERT base en su versión cased (sí diferencia entre mayúsculas y minúsculas), y lo vamos a utilizar para tokenizar distintas oraciones.

Para cargar el tokenizador, en lugar de usar la clase **AutoTokenizer*, en esta ocasión, usaremos la clase propia de BERT pra cargar su tokenizador (ambas clases hacen exactamente lo mismo):


In [ ]:
from transformers import BertTokenizer
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

En la siguiente oración, vemos que la palabra 'Pharmaceutical' ha sido dividida en varios morfemas ('Ph', '##arma', '##ce', '##utical'), que van a corresponder con los siguientes ids 7642, 24275, 2093, 19748:

In [ ]:
text = "Pharmaceutical means connected with the industrial production of medicines"
print(bert_tokenizer.tokenize(text))
print(bert_tokenizer(text).input_ids)

['Ph', '##arma', '##ce', '##utical', 'means', 'connected', 'with', 'the', 'industrial', 'production', 'of', 'medicines']
[101, 7642, 24275, 2093, 19748, 2086, 3387, 1114, 1103, 3924, 1707, 1104, 26016, 102]


Vamos a decodificar esta lista $[7642, 24275, 2093, 19748]$, y comprobar que en efecto, devuelve los morfemas $['Ph', '##arma', '##ce', '##utical']$
:

In [ ]:
bert_tokenizer.convert_ids_to_tokens([7642, 24275, 2093, 19748])

['Ph', '##arma', '##ce', '##utical']

Prueba con las siguientes oraciones y observa la tokenización:

In [ ]:
text='An anachronism is something (or someone) that is out of place in terms of time or chronology'
bert_tokenizer.tokenize(text)

['An',
 'an',
 '##ach',
 '##ron',
 '##ism',
 'is',
 'something',
 '(',
 'or',
 'someone',
 ')',
 'that',
 'is',
 'out',
 'of',
 'place',
 'in',
 'terms',
 'of',
 'time',
 'or',
 'ch',
 '##ronology']

In [ ]:
text='Accismus is a form of irony in which someone feigns indifference to something he or she desires'
bert_tokenizer.tokenize(text)

['A',
 '##cci',
 '##smus',
 'is',
 'a',
 'form',
 'of',
 'irony',
 'in',
 'which',
 'someone',
 'f',
 '##ei',
 '##gn',
 '##s',
 'in',
 '##di',
 '##fference',
 'to',
 'something',
 'he',
 'or',
 'she',
 'desires']

In [ ]:
text='A cacophony is a harsh mixture of sounds'
bert_tokenizer.tokenize(text)

['A', 'ca', '##co', '##phony', 'is', 'a', 'harsh', 'mixture', 'of', 'sounds']

In [ ]:
text = 'Many filmmakers try for some kind of verisimilitude in their stories'
bert_tokenizer.tokenize(text)

['Many',
 'filmmakers',
 'try',
 'for',
 'some',
 'kind',
 'of',
 've',
 '##ris',
 '##im',
 '##ili',
 '##tude',
 'in',
 'their',
 'stories']

Vamos a probar también con textos en español, para ello volvemos a cargar el modelo multilingüe de BERT:

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")


In [ ]:
text='El metaverso es una realidad digital.'
bert_tokenizer.tokenize(text)

['el', 'meta', '##verso', 'es', 'una', 'realidad', 'digital', '.']

In [ ]:
bert_tokenizer(text).input_ids

[101, 10117, 16650, 78426, 10153, 10155, 35781, 10764, 119, 102]

In [ ]:
bert_tokenizer.convert_ids_to_tokens([16650, 78426])

['meta', '##verso']